In [ ]:
%load_ext autoreload
%autoreload 2
import sys, warnings, time, numpy, yaml
sys.path.append("../src/") # go to parent dir
from data_access import get_X, get_y, get_train_test
from models.factory import ModelFactory
from models.xgboost_model import XgboostModel
warnings.filterwarnings('ignore')

In [ ]:
train_data = get_X('train')
train_scores = get_y()
test_data = get_X('test')
X_train, y_train, X_test, y_test, X_valid, y_valid, target = get_train_test(train_size=0.8, random_state=42)

In [ ]:
%load_ext autoreload
%autoreload 2
model = XgboostModel(X_train, y_train, X_valid, y_valid, train_scores)
model.train()
model.evaluate(X_test)

In [ ]:
params_by_name={
    'benchmark2': {
          'booster': 'gbtree',
          'tree_method':'hist',
          'max_depth': 8, 
          'learning_rate': 0.025,
          'objective': 'multi:softprob',
          'num_class': 2,
          'eval_metric':'mlogloss'
        },
    'benchmark3': {
          'booster': 'gblinear',
          'tree_method':'hist',
          'max_depth': 8, 
          'learning_rate': 0.025,
          'objective': 'multi:softprob',
          'num_class': 2,
          'eval_metric':'mlogloss'
        },
}
for name, params in params_by_name.items():
    model = XgboostModel(X_train, y_train, X_valid, y_valid, train_scores, params)
    model.train()
    model.evaluate(X_test)
    model.name = name
    model.save(test_data)

In [ ]:
%load_ext autoreload
%autoreload 2
def get_params(booster, tree_method, eval_metric='mlogloss'):
    return {
          'booster': booster,
          'tree_method': tree_method,
          'max_depth': 8, 
          'learning_rate': 0.025,
          'objective': 'multi:softprob',
          'num_class': 2,
          'eval_metric':eval_metric
        }
# boosters = ['gbtree', 'gblinear', 'dart']
boosters = ['gblinear']
# tree_methods = ['auto', 'exact', 'approx', 'hist']
tree_methods = ['hist']
# eval_metrics = ['mphe', 'merror', 'mlogloss', 'auc']
eval_metrics = ['mlogloss']
for booster in boosters:
    for tree_method in tree_methods:
        for eval_metric in eval_metrics:
          start = time.time()
          name = f'{booster}_{tree_method}_{eval_metric}'
          params = get_params(booster, tree_method, eval_metric)
          model = XgboostModel(X_train, y_train, X_valid, y_valid, train_scores, params)
          model.train()
          score = model.evaluate(X_test)
          end = time.time()
          print(f'{name}={score} in {numpy.round((end-start), 2)}s')

In [ ]:
%load_ext autoreload
%autoreload 2
with open('../confs/models.yaml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, X_train, y_train, X_valid, y_valid, train_scores)
for model in factory.get_models():
    start = time.time()
    model.train()
    score = model.evaluate(X_test)
    end = time.time()
    print(f'{name}={score} in {numpy.round((end-start), 2)}s')